In [1]:
# %load_ext autoreload
# %autoreload 2

In [1]:
import os
import sys
cm_folder = "/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/notebooks/Causal Model/"
src_path = os.path.join(cm_folder, "src")
sys.path.append(src_path)

In [2]:
from typing import Any

import dill
import numpy as np

from CrossValidation import cross_validation
from structured_perceptron import StructuredPerceptron
from Settings import Settings

from window_based_tagger_config import get_config
from crel_helper import get_cr_tags
from crel_processing import essay_to_crels_cv
from evaluation import evaluate_model_essay_level, get_micro_metrics, metrics_to_df
from feature_normalization import min_max_normalize_feats
from function_helpers import get_function_names
from results_procesor import ResultsProcessor
from train_parser import essay_to_crels, create_extractor_functions
from cost_functions import micro_f1_cost_plusepsilon
from train_reranker import train_model, train_instance, get_essays_for_data, evaluate_ranker
from searn_parser_breadth_first import SearnModelBreadthFirst
from causal_model_features import CausalModelType
from feature_extraction import get_features_from_probabilities
from results_procesor import ResultsProcessor
from filter_features import filter_feats

from wordtagginghelper import merge_dictionaries
from results_procesor import ResultsProcessor, __MICRO_F1__
from evaluation import add_cr_labels

from random import shuffle
from joblib import Parallel, delayed
from collections import defaultdict

In [3]:
def train_model_fold(xs_train, xs_test, name2essay, learning_rate, max_update_items, set_cr_tags,\
                     initial_weight, max_epochs, early_stop_iters):

    mdl = StructuredPerceptron(
        learning_rate=learning_rate, max_update_items=max_update_items, initial_weight=initial_weight)

    return train_model(mdl, xs_train=xs_train, xs_test=xs_test, name2essay=name2essay,
            max_epochs=max_epochs, early_stop_iters=early_stop_iters, set_cr_tags=set_cr_tags,
            train_instance_fn=train_instance,
            verbose=False, return_metrics=True, early_stopping=False)

def train_model_parallel(cv_folds, name2essay, learning_rate, max_update_items, set_cr_tags, \
                         initial_weight, max_epochs=5, early_stop_iters=5, n_jobs=None):

    if n_jobs == None:
        n_jobs = len(cv_folds)
    try:
        results = Parallel(n_jobs=n_jobs)(
            delayed(train_model_fold)(train, test, name2essay, learning_rate, max_update_items, set_cr_tags, \
                                      initial_weight, max_epochs, early_stop_iters)
            for (train, test) in cv_folds)

        f1s = []
        for tpl in results:
            best_test_f1, best_iterations, train_ys_bytag, train_pred_ys_bytag, test_ys_bytag, test_pred_ys_bytag, num_feats = tpl
            f1s.append(best_test_f1)
        return np.mean(f1s)
    except KeyboardInterrupt:
        print("Process stopped by user")

def train_model_parallel_logged(training_collection_name: str, results_processor: ResultsProcessor,
                                feat_extractors, params,
                                cv_folds, name2essay,
                                learning_rate: float, max_update_items:int, set_cr_tags, \
                                initial_weight: float,  max_epochs=5, early_stop_iters=5, n_jobs=None):
    if not n_jobs or n_jobs == None:
        n_jobs = len(cv_folds)

    try:
        results = Parallel(n_jobs=n_jobs)(
            delayed(train_model_fold)(train, test, name2essay, learning_rate, max_update_items, set_cr_tags, \
                                      initial_weight, max_epochs, early_stop_iters)
            for (train, test) in cv_folds)

        cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag = defaultdict(list), defaultdict(list)
        cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag = defaultdict(list), defaultdict(list)

        f1s = []
        feats = []
        for tpl in results:
            best_test_f1, best_iterations, train_ys_bytag, train_pred_ys_bytag, test_ys_bytag, test_pred_ys_bytag, num_feats = tpl
            feats.append(num_feats)
            f1s.append(best_test_f1)

            merge_dictionaries(train_ys_bytag, cv_sent_td_ys_by_tag)
            merge_dictionaries(test_ys_bytag, cv_sent_vd_ys_by_tag)

            merge_dictionaries(train_pred_ys_bytag, cv_sent_td_predictions_by_tag)
            merge_dictionaries(test_pred_ys_bytag, cv_sent_vd_predictions_by_tag)


        ALGO = "Structured Perceptron"
        validation_collection = training_collection_name.replace("_TD", "_VD")

        # extractors = list(map(lambda fn: fn.func_name, feat_extractors))
        extractors = list(feat_extractors)

        parameters = {
            "learning_rate":        learning_rate,
            "loss_type":            "None",
            "max_update_items":     max_update_items,
            "initial_weight":       initial_weight,

            "max_epochs":           max_epochs,
            "early_stopping_iters": early_stop_iters,

            "extractors":           extractors,

            # Add in number of features
            "num_feats_per_fold":   feats,
            "num_feats_MEAN":       np.mean(feats)
        }
        # add in additional parameters not passed in
        parameters.update(params)

        wd_td_objectid = results_processor.persist_results(training_collection_name,
                                                           cv_sent_td_ys_by_tag,
                                                           cv_sent_td_predictions_by_tag,
                                                           parameters, ALGO)

        wd_vd_objectid = results_processor.persist_results(validation_collection,
                                                           cv_sent_vd_ys_by_tag,
                                                           cv_sent_vd_predictions_by_tag,
                                                           parameters, ALGO)

        avg_f1 = float(results_processor.get_metric(validation_collection, wd_vd_objectid, __MICRO_F1__)["f1_score"])
        return avg_f1
    except KeyboardInterrupt:
        print("Process stopped by user")

In [6]:
# Data Set Partition
CV_FOLDS = 5
MIN_FEAT_FREQ = 5

# Global settings
settings = Settings()
CAUSAL_MODEL_TYPE = CausalModelType.SKIN_CANCER
root_folder = settings.data_directory + "SkinCancer/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"

crels_folder = "./crels/SC"
coref_root = root_folder + "CoReference/"
coref_output_folder = coref_root + "CRel/"

MONGO_COLLECTION = "SC_STR_PCPTRN_RE-RANKER_FEATURE_SEL_TD_2"

config = get_config(training_folder)
results_processor = ResultsProcessor(dbname="metrics_causal_model_reranker")

train_fname = coref_output_folder + "training_crel_anatagged_essays_most_recent_code.dill"
with open(train_fname, "rb") as f:
    pred_tagged_essays_train = dill.load(f)

test_fname = coref_output_folder + "test_crel_anatagged_essays_most_recent_code.dill"
with open(test_fname, "rb") as f:
    pred_tagged_essays_test = dill.load(f)

print(len(pred_tagged_essays_train), len(pred_tagged_essays_test))

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
870 218


In [7]:
cr_tags = get_cr_tags(train_tagged_essays=pred_tagged_essays_train, tag_essays_test=pred_tagged_essays_test)

set_cr_tags = set(cr_tags)
list(set_cr_tags)[0:10]

['Causer:2->Result:50',
 'Causer:3->Result:12',
 'Causer:3->Result:2',
 'Causer:2->Result:2',
 'Causer:12->Result:4',
 'Causer:50->Result:1',
 'Causer:1->Result:3',
 'Causer:4->Result:50',
 'Causer:3->Result:6',
 'Causer:4->Result:4']

In [8]:
all_essays = pred_tagged_essays_train + pred_tagged_essays_test
name2essay = {}
for essay in all_essays:
    name2essay[essay.name] = essay

name2crels = essay_to_crels(all_essays, set_cr_tags)
assert len(name2crels) == len(all_essays)

# Train Re-Ranker

## Extract Parses from Sentence Parser

In [9]:
best_top_n = 5 # 2
min_feat_freq = 1
best_max_upd = 2
best_max_parses = 300
best_min_prob = 0.0  # min prob of 0 seems better

In [10]:
rr_fname = "xs_rerank_" + str(best_top_n) + ".dill"
with open(os.path.join(crels_folder, rr_fname), "rb") as f:
    xs_rerank = dill.load(f)

rr_fname = "xs_rerank_test" + str(best_top_n) + ".dill"
with open(os.path.join(crels_folder, rr_fname), "rb") as f:
    xs_test_rerank = dill.load(f)
    
assert len(xs_rerank) == len(pred_tagged_essays_train),     "Wrong number of train crels"
assert len(xs_test_rerank) == len(pred_tagged_essays_test), "Wrong number of test crels"
len(xs_rerank), len(xs_test_rerank)

(870, 218)

## Prepare Features

In [11]:
%%time
xs = get_features_from_probabilities(xs_rerank, name2crels, best_max_parses, 
                                     causal_model_type=CAUSAL_MODEL_TYPE,
                                     min_feat_freq=min_feat_freq, min_prob=best_min_prob)

cv_folds_rerank = cross_validation(xs, 5)
cv_folds_mm = [min_max_normalize_feats(train, test) for (train, test) in cv_folds_rerank]

CPU times: user 19min 14s, sys: 46.5 s, total: 20min 1s
Wall time: 52min 25s


In [12]:
%%time
xs_test = get_features_from_probabilities(xs_test_rerank, name2crels, best_max_parses, 
                                          causal_model_type=CAUSAL_MODEL_TYPE,
                                          min_feat_freq=min_feat_freq, min_prob=best_min_prob)

CPU times: user 34.5 s, sys: 919 ms, total: 35.4 s
Wall time: 35.9 s


In [13]:
# Prepare test dataset 
  # training data comes from the test fold predictions from CV on the full training dataset
xs_train = []
for train, test in cv_folds_rerank:
    xs_train.extend(test)

# Normalize both using training data
xs_train_mm, xs_test_mm = min_max_normalize_feats(xs_train,xs_test)

### Initial Parameters

In [14]:
learning_rate = 0.3 # 0.3
max_update_items = 2  # best_max_upd - 2
initial_weight = 0.01  # was 0.01

## Train on Test Data

### Tuning Dataset

In [15]:
best_top_n, learning_rate

(5, 0.3)

In [16]:
num_train = int(0.8 * len(xs_train_mm))
tmp_train_copy = list(xs_train_mm)
np.random.shuffle(tmp_train_copy)
tmp_train, tmp_test = tmp_train_copy[:num_train], tmp_train_copy[num_train:]

### Determine Number of Training Iterations

In [17]:
%%time
# use training data to determine number of iterations
mdl = StructuredPerceptron(learning_rate=learning_rate,
                        max_update_items=max_update_items, initial_weight=initial_weight)
# Determine number of training iterations
best_mdl, test_acc_df_ml, best_iterations = train_model(mdl, xs_train=tmp_train, xs_test=tmp_test, name2essay=name2essay, set_cr_tags=set_cr_tags,
     max_epochs=20, early_stop_iters=5, 
     train_instance_fn = train_instance, verbose=True, early_stopping=True)
print("Best iterations:", best_iterations)

Epoch: 0 Train Accuracy: 0.8134 Test Accuracy: 0.8053
Epoch: 1 Train Accuracy: 0.8173 Test Accuracy: 0.8035
Epoch: 2 Train Accuracy: 0.8198 Test Accuracy: 0.8045
Epoch: 3 Train Accuracy: 0.8204 Test Accuracy: 0.8035
Epoch: 4 Train Accuracy: 0.8219 Test Accuracy: 0.8048
Epoch: 5 Train Accuracy: 0.8226 Test Accuracy: 0.8019
Best Test Acc: 0.8053
Best iterations: 1
CPU times: user 11min 29s, sys: 10.7 s, total: 11min 39s
Wall time: 11min 52s


# Feature Selection

In [18]:
from filter_features import filter_feats

prefixes = [
    "Prob-",
#     "CREL_Pair-",
    "Inv-",
    "num_crels",
    "Tally-",
    "CChain-",
    "CChainStats-",
    "Above-",
    "CREL_",
    "Propn_",
    "Diff_"
]
# xs_fltr_train, xs_fltr_test = filter_feats(xs_train_mm, xs_test_mm, prefixes)
assert len(prefixes) == len(set(prefixes)), "Duplicate prefixes found"

In [19]:
best_f1 = -1
current_best = []
remaining = list(prefixes)

In [20]:
# best_iterations = 1

In [21]:
MONGO_COLLECTION, best_iterations, initial_weight

('SC_STR_PCPTRN_RE-RANKER_FEATURE_SEL_TD_2', 1, 0.01)

In [ ]:
%%time
params = {
    "learning_rate":   learning_rate,
    "best_max_upd":    best_max_upd,
    "best_max_parses": best_max_parses,
    "best_min_prob":   best_min_prob,
    "min_feat_freq":   min_feat_freq,
    "best_iterations": best_iterations,
    "best_top_n":      best_top_n
}

print("Starting...")
while True:
    if len(remaining) == 0:
        break
    
    f1_by_prefix = dict()
    for prefix in remaining:
        new_prefixes = current_best + [prefix]
        
        cv_filtered = []
        for tr, test in cv_folds_mm:
            x_tr,x_test = filter_feats(tr, test, new_prefixes)
            cv_filtered.append((x_tr,x_test))
        
        f1_by_prefix[prefix] = train_model_parallel_logged(
            n_jobs=1,
            training_collection_name=MONGO_COLLECTION, results_processor=results_processor,
            feat_extractors=new_prefixes, params=params,
            cv_folds=cv_filtered, name2essay=name2essay,
            learning_rate=learning_rate, max_update_items=best_max_upd, 
            set_cr_tags=set_cr_tags, initial_weight=initial_weight,
            # use best iterations from above
            max_epochs=best_iterations, early_stop_iters=best_iterations
        )
        print("\t{length} feats F1: {f1:.6f} Prefixes: {prefixes}".format(
            length=len(new_prefixes), f1=f1_by_prefix[prefix], prefixes=str(new_prefixes)))
    
    best_prefix, new_best_f1 = sorted(f1_by_prefix.items(), key = lambda tpl: -tpl[1])[0]
    if new_best_f1 > best_f1:
        best_f1 = new_best_f1
        current_best.append(best_prefix)
        remaining.remove(best_prefix)
        print("{length} feats, new Best F1: {f1:.6f} Prefixes: {prefixes}".format(
            length=len(current_best), f1=best_f1, prefixes=str(current_best)))
    else:
        print("No further improvement, stopping")
        break

# 1 feats, new Best F1: 0.7430 Prefixes: ['CREL_']
# 2 feats, new Best F1: 0.7438 Prefixes: ['CREL_', 'Prob-']
# 3 feats, new Best F1: 0.7461 Prefixes: ['CREL_', 'Prob-', 'CChainStats-']
# No further improvement, stopping 

Starting...
	1 feats F1: 0.803292 Prefixes: ['Prob-']
	1 feats F1: 0.446486 Prefixes: ['Inv-']
	1 feats F1: 0.672472 Prefixes: ['num_crels']
	1 feats F1: 0.789541 Prefixes: ['Tally-']
	1 feats F1: 0.613051 Prefixes: ['CChain-']
	1 feats F1: 0.622785 Prefixes: ['CChainStats-']
	1 feats F1: 0.809840 Prefixes: ['Above-']
	1 feats F1: 0.798436 Prefixes: ['CREL_']
	1 feats F1: 0.670684 Prefixes: ['Propn_']
	1 feats F1: 0.629101 Prefixes: ['Diff_']
1 feats, new Best F1: 0.809840 Prefixes: ['Above-']
	2 feats F1: 0.809181 Prefixes: ['Above-', 'Prob-']
	2 feats F1: 0.810370 Prefixes: ['Above-', 'Inv-']
	2 feats F1: 0.810019 Prefixes: ['Above-', 'num_crels']
	2 feats F1: 0.809492 Prefixes: ['Above-', 'Tally-']
	2 feats F1: 0.807107 Prefixes: ['Above-', 'CChain-']
	2 feats F1: 0.807997 Prefixes: ['Above-', 'CChainStats-']
	2 feats F1: 0.802278 Prefixes: ['Above-', 'CREL_']
	2 feats F1: 0.809365 Prefixes: ['Above-', 'Propn_']
	2 feats F1: 0.809651 Prefixes: ['Above-', 'Diff_']
2 feats, new Best F

In [ ]:
current_best, best_f1, best_iterations

## Apply to Test Data

In [ ]:
xs_train_mm_fltr, xs_test_mm_fltr = filter_feats(xs_train_mm, xs_test_mm, current_best) 

In [ ]:
# num_train = int(0.8 * len(xs_train_mm_fltr))
# tmp_train_copy = list(xs_train_mm_fltr)
# np.random.shuffle(tmp_train_copy)
# tmp_train, tmp_test = tmp_train_copy[:num_train], tmp_train_copy[num_train:]

## Determine the Optimal Number of Training Iterations

In [ ]:
# initial_weight = 0

In [ ]:
# %%time
# mdl = StructuredPerceptron(learning_rate=learning_rate,
#                         max_update_items=max_update_items, initial_weight=initial_weight)
# # Determine number of training iterations
# best_mdl, test_acc_df_ml, best_iterations = train_model(mdl, xs_train=tmp_train, 
#     xs_test=tmp_test, name2essay=name2essay, set_cr_tags=set_cr_tags,
#      max_epochs=20, early_stop_iters=3, train_instance_fn = train_instance,
#                                                         verbose=True,  early_stopping=True)

In [ ]:
# best_iterations

## Run for X Iterations on the Test Dataset

In [ ]:
mdl = StructuredPerceptron(learning_rate=learning_rate,
                        max_update_items=max_update_items, initial_weight=initial_weight)

best_mdl, test_acc_df_ml,_ = train_model(mdl,  
    xs_train=xs_train_mm_fltr, xs_test=xs_test_mm_fltr,
    name2essay=name2essay, set_cr_tags=set_cr_tags,
    max_epochs=best_iterations, early_stop_iters=best_iterations,
    train_instance_fn = train_instance, verbose=True, early_stopping=False)

# Best Test Acc: 0.7516
best_iterations

In [ ]:
initial_weight

In [ ]:
test_acc_df_ml

In [ ]:
sorted(best_mdl.weights.items(), key = lambda tpl: -abs(tpl[1]))[0:50]

## Count Features

In [31]:
MONGO_COLLECTION

'SC_STR_PCPTRN_RE-RANKER_FEATURE_SEL_TD_2'

In [41]:
new_prefixes = list(prefixes) # all features

cv_filtered = []
for tr, test in cv_folds_mm:
    x_tr,x_test = filter_feats(tr, test, new_prefixes)
    cv_filtered.append((x_tr,x_test))
        
train_model_parallel_logged(
            training_collection_name=MONGO_COLLECTION + "_MAX_FT_CNTS", results_processor=results_processor,
            feat_extractors=new_prefixes, params=params,
            n_jobs=1,
            cv_folds=cv_filtered, name2essay=name2essay, C=best_C, 
            pa_type=1, loss_type="ml", max_update_items=best_max_upd, 
            set_cr_tags=set_cr_tags, initial_weight=initial_weight,
            # use best iterations from above
            max_epochs=1, early_stop_iters=1
)

0.7430529120669965